# Multi-Qubit Gate Stuff

In [1]:
import numpy as np
from scipy.linalg import expm, sqrtm, svdvals
from scipy.optimize import minimize

import cirq

import sys
sys.path.append("../../Gates_Lab_Suite")

from Auto_Algorithm import *
from Core_Definition import *
from Visualization import *

np.set_printoptions(precision=3)

### Native Gates

In [2]:
def Ra(theta, phi):

    theta = theta * np.pi
    phi = phi * np.pi

    return np.array([[np.cos(theta / 2), -1j * np.exp(-1j * phi) * np.sin(theta / 2)], 
                     [-1j * np.exp(1j * phi) * np.sin(theta / 2), np.cos(theta / 2)]])

def Rz(theta):

    theta = theta * np.pi 

    return np.array([[np.exp(-1j * theta / 2), 0],
                     [0, np.exp(1j * theta / 2)]])


def Ry(theta):
    return Ra(theta, 0.5)


def Rx(theta):
    return Ra(theta, 0)


Id = np.array([[1, 0], [0, 1]])


def XX(xi):

    xi = xi * np.pi
    
    return np.array([[np.cos(xi), 0, 0, -1j*np.sin(xi)], 
                     [0, np.cos(xi), -1j*np.sin(xi), 0], 
                     [0, -1j*np.sin(xi), np.cos(xi), 0], 
                     [-1j*np.sin(xi), 0, 0, np.cos(xi)]])

# Common Non-Native Gates

In [3]:
def CNOT(): 
    
    return np.array([[1, 0, 0, 0],
                     [0, 1, 0, 0],
                     [0, 0, 0, 1],
                     [0, 0, 1, 0]])


def gates_circuit_CNOT():

    m = Quantum_Gate("CNOT", 0, 1).Matrix_Representation(2)
    return m


def figgatt_circuit_CNOT_method1(xi=0.25, theta1=0.5, theta2=0.5):

    circuit = np.kron(Ry(0.5), Id)
    circuit = np.matmul(XX(xi), circuit)
    circuit = np.matmul(np.kron(Rx(-theta1), Rx(-theta2)), circuit)
    circuit = np.matmul(np.kron(Ry(-0.5), Id), circuit)

    return circuit


def figgatt_circuit_CNOT_method2(xi=0.25, theta1=0.5, theta2=0.5):

    circuit = np.kron(Ry(-0.5), Id)
    circuit = np.matmul(XX(xi), circuit)
    circuit = np.matmul(np.kron(Ry(theta1), Rx(theta2)), circuit)
    circuit = np.matmul(np.kron(Rz(1), Id), circuit)

    return circuit


def CZ():

    return np.array([[1, 0, 0, 0],
                     [0, 1, 0, 0],
                     [0, 0, 1, 0],
                     [0, 0, 0, -1]])


def gates_circuit_CZ():

    m = Quantum_Gate("CZ", 0, 1).Matrix_Representation(2)

    return m


def figgatt_circuit_CZ():

    circuit = np.kron(Ry(-0.5), Ry(0.5))
    circuit = np.matmul(np.kron(Rx(-0.5), Rx(0.5)), circuit)
    circuit = np.matmul(XX(0.25), circuit)
    circuit = np.matmul(np.kron(Ry(0.5), Ry(-0.5)), circuit)

    return circuit


def CRx(theta):

    theta *= np.pi

    return np.array([[1, 0, 0, 0],
                     [0, 1, 0, 0],
                     [0, 0, np.cos(theta/2), -1j * np.sin(theta/2)],
                     [0, 0, -1j * np.sin(theta/2), np.cos(theta/2)]])


def CRy(theta):

    theta *= np.pi

    return np.array([[1, 0, 0, 0],
                     [0, 1, 0, 0],
                     [0, 0, np.cos(theta/2), -np.sin(theta/2)],
                     [0, 0, np.sin(theta/2), np.cos(theta/2)]])

# Distance Measures

In [4]:
def check_unitary_equality(U, V):

    return cirq.equal_up_to_global_phase(U, V)


def fidelity(U, V):
  """
  Calculates the fidelity between two density matrices.

  Args:
    rho: First density matrix (NumPy array).
    sigma: Second density matrix (NumPy array).

  Returns:
    The fidelity value (float).
  """
  
  # Calculate the square roots of the density matrices
  rho_sqrt = sqrtm(U)
  sigma_sqrt = sqrtm(V)

  # Calculate the product of the square roots
  product = rho_sqrt @ sigma_sqrt

  # Calculate the singular values of the product
  singular_values = svdvals(product)

  # Calculate the trace norm (sum of singular values)
  trace_norm = singular_values.sum()

  # Square the trace norm to get the fidelity
  fidelity = trace_norm**2

  return fidelity


def spectral_distance(U, V):
   
   return np.linalg.norm(U-V, 2)

# Seems to be the best so far:
def spectral_distance_ignoring_global_phase(U, V):
    
    phase = np.angle(np.trace(np.dot(U.conj().T, V)))
    V_phase_corrected = V * np.exp(-1j * phase)

    return spectral_distance(U, V_phase_corrected)

In [5]:
figgatt_circuit_CNOT_method1()

array([[ 7.071e-01+7.071e-01j,  6.687e-33+1.112e-32j,
         9.813e-17+5.888e-17j, -7.716e-17-3.386e-17j],
       [ 6.687e-33+1.112e-32j,  7.071e-01+7.071e-01j,
        -7.716e-17-3.386e-17j,  9.813e-17+5.888e-17j],
       [-2.165e-17+7.716e-17j, -3.925e-17-1.178e-16j,
         1.766e-16-1.374e-16j,  7.071e-01+7.071e-01j],
       [-3.925e-17-1.178e-16j, -2.165e-17+7.716e-17j,
         7.071e-01+7.071e-01j,  1.766e-16-1.374e-16j]])

In [6]:
spectral_distance_ignoring_global_phase(figgatt_circuit_CNOT_method1(), (CRx(1) @ np.kron(Rz(0.5), Id)))

4.4209952906199435e-16

# Optimization

### Naive ansatz

Sandwich an XX gate with arbitrary single qubit rotations using Euler rotations

In [7]:
def circuit_ansatz_euler_rotations(alpha1, beta1, gamma1, alpha2, beta2, gamma2, alphap1, betap1, gammap1, alphap2, betap2, gammap2, xi_xx):
    
    # Arbitrary single-qubit rotation on both qubits (each qubit can have different angle)
    circuit = np.kron(Id, Id)
    circuit = np.matmul(np.kron(Rz(alpha1), Rz(alpha2)), circuit)
    circuit = np.matmul(np.kron(Ry(beta1), Ry(beta2)), circuit)
    circuit = np.matmul(np.kron(Rz(gamma1), Rz(gamma2)), circuit)

    # XX gate of any angle
    circuit = np.matmul(XX(xi_xx), circuit)

    # Arbitrary single-qubit rotation on both qubits (each qubit can have different angle)
    circuit = np.matmul(np.kron(Rz(alphap1), Rz(alphap2)), circuit)
    circuit = np.matmul(np.kron(Ry(betap1), Ry(betap2)), circuit)
    circuit = np.matmul(np.kron(Rz(gammap1), Rz(gammap2)), circuit)

    return circuit

In [8]:
def figgatt_circuit_CNOT_method1(xi=0.25, theta1=0.5, theta2=0.5):

    circuit = np.kron(Id, Id)
    circuit = np.matmul(np.kron(Ry(0.5), Id), circuit)
    circuit = np.matmul(XX(xi), circuit)
    circuit = np.matmul(np.kron(Rx(-theta1), Rx(-theta2)), circuit)
    circuit = np.matmul(np.kron(Ry(-0.5), Id), circuit)

    return circuit

In [9]:
bounds = ((-2, 2), (-2, 2), (-2, 2), (-2, 2), (-2, 2), (-2, 2), (-2, 2), (-2, 2), (-2, 2), (-2, 2), (-2, 2), (-2, 2), (-2, 2))
initial_guess = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.25]

Euler rotations for CNOT:

In [10]:
exact_solution_CNOT = [0, 0.5, 0, 0, 0, 0, -1, 0.5, 0.5, -0.5, 0.5, 0.5, 0.25]

In [11]:
spectral_distance_ignoring_global_phase(circuit_ansatz_euler_rotations(*exact_solution_CNOT), figgatt_circuit_CNOT_method1()) 

3.97757079643251e-16

Euler rotations for CZ:

In [12]:
exact_solution_CZ = [0, 0.5, 0, 0, 0, 0, 0, -0.5, -0.5, 0.5, 0.5, -0.5, 0.25]

In [13]:
CZ()

array([[ 1,  0,  0,  0],
       [ 0,  1,  0,  0],
       [ 0,  0,  1,  0],
       [ 0,  0,  0, -1]])

In [14]:
circuit_ansatz_euler_rotations(*exact_solution_CZ)

array([[ 7.716e-17+1.449e-16j,  7.071e-01-7.071e-01j,
        -7.716e-17-3.386e-17j,  5.551e-17-5.551e-17j],
       [ 7.071e-01-7.071e-01j,  1.449e-16+7.716e-17j,
         5.551e-17-5.551e-17j, -7.716e-17-3.386e-17j],
       [-5.551e-17+5.551e-17j, -3.386e-17-7.716e-17j,
         7.071e-01-7.071e-01j, -3.386e-17+3.386e-17j],
       [-3.386e-17-7.716e-17j, -5.551e-17+5.551e-17j,
         3.386e-17-3.386e-17j,  7.071e-01-7.071e-01j]])

### Objective functions

In [15]:
def CNOT_optimization_function(x):

    return spectral_distance_ignoring_global_phase(circuit_ansatz_euler_rotations(*x), CNOT())



def CZ_optimization_function(x):

    return spectral_distance_ignoring_global_phase(circuit_ansatz_euler_rotations(*x), CZ())

In [16]:
def get_CRx_gate(theta):


    def CRx_optimization_function(x):

        return spectral_distance_ignoring_global_phase(circuit_ansatz_euler_rotations(*x), CRx(theta))

    result = minimize(CRx_optimization_function, initial_guess, bounds=bounds, method="L-BFGS-B")

    clean_result = []
    for r in result.x:
        if r < 1e-5:
            clean_result.append(0)
        else:
            clean_result.append(r)
    clean_result

    return circuit_ansatz_euler_rotations(*clean_result)


def get_CRy_gate(theta):


    def CRy_optimization_function(x):

        return spectral_distance_ignoring_global_phase(circuit_ansatz_euler_rotations(*x), CRy(theta))

    result = minimize(CRy_optimization_function, initial_guess, bounds=bounds, method="L-BFGS-B")

    clean_result = []
    for r in result.x:
        if r < 1e-5:
            clean_result.append(0)
        else:
            clean_result.append(r)
    clean_result

    return circuit_ansatz_euler_rotations(*clean_result)

In [17]:
theta = 0.245
spectral_distance_ignoring_global_phase(get_CRx_gate(theta), CRx(theta))

1.4761445386105821e-07

# Try using multiple controls...

In [18]:
def CCNot(theta):

    theta *= np.pi

    return np.array([[1, 0, 0, 0, 0, 0, 0, 0],
                     [0, 1, 0, 0, 0, 0, 0, 0],
                     [0, 0, 1, 0, 0, 0, 0, 0],
                     [0, 0, 0, 1, 0, 0, 0, 0],
                     [0, 0, 0, 0, 1, 0, 0, 0],
                     [0, 0, 0, 0, 0, 1, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 1],
                     [0, 0, 0, 0, 0, 0, 1, 0]])

def CRx(theta):

    theta *= np.pi

    return np.array([[1, 0, 0, 0],
                     [0, 1, 0, 0],
                     [0, 0, np.cos(theta/2), -1j * np.sin(theta/2)],
                     [0, 0, -1j * np.sin(theta/2), np.cos(theta/2)]])

In [19]:
def circuit_ansatz_euler_rotations_three_qubits(alpha1, beta1, gamma1, alpha2, beta2, gamma2, alphap1, betap1, gammap1, alphap2, betap2, gammap2, xi_xx):
    
    # Arbitrary single-qubit rotation on both qubits (each qubit can have different angle)
    circuit = np.kron(Id, Id)
    circuit = np.matmul(np.kron(Rz(alpha1), Rz(alpha2)), circuit)
    circuit = np.matmul(np.kron(Ry(beta1), Ry(beta2)), circuit)
    circuit = np.matmul(np.kron(Rz(gamma1), Rz(gamma2)), circuit)

    # XX gate of any angle
    circuit = np.matmul(XX(xi_xx), circuit)

    # Arbitrary single-qubit rotation on both qubits (each qubit can have different angle)
    circuit = np.matmul(np.kron(Rz(alphap1), Rz(alphap2)), circuit)
    circuit = np.matmul(np.kron(Ry(betap1), Ry(betap2)), circuit)
    circuit = np.matmul(np.kron(Rz(gammap1), Rz(gammap2)), circuit)

    return circuit